# Problema 2 - Linha de Produção
Esse problema pode ser representado como um **Problema do Caminho Crítico**

In [29]:
import networkx as nx
import matplotlib.pyplot as plt
import sys

In [30]:
# Enumeração Primitiva -- Só facilita a leitura
S = 0
A = 1
B = 2
C = 3
D = 4
E = 5
F = 6
G_v = 7
H = 8
I = 9
J = 10
K = 11
L = 12

In [31]:
# Criando nosso dígrafo ponderado
G = nx.DiGraph()
G.add_edges_from(
    [
        (S, C,     {"weight": 10}),
        (S, A,     {"weight": 8}),
        (S, D,     {"weight": 12}),
        (S, B,     {"weight": 8}),
        (C, E,     {"weight": 8}),
        (C, F,     {"weight": 11}),
        (A, E,     {"weight": 8}),
        (A, F,     {"weight": 11}),
        (A, G_v,   {"weight": 15}),
        (D, G_v,   {"weight": 15}),
        (B, L,     {"weight": 7}),
        (E, H,     {"weight": 9}),
        (F, I,     {"weight": 7}),
        (G_v, J,   {"weight": 4}),
        (H, J,     {"weight": 4}),
        (J, K,     {"weight": 6}),
        (K, L,     {"weight": 7}),
        (I, K,     {"weight": 6}),  
    ]
)

## Calculando o Caminho Crítico e seu peso
Vamos usar as funções abaixo que implementam o Dijktra com pesos negativos. O Dijkstra funciona pois o grafo é um Dígrafo Acíclico Ponderado, o que não gera ciclos negativos.

In [44]:
# Cria um grafo cópia que vai ser entrada de Dijkstra com pesos invertidos
G2 = G.copy()

# Muda os pesos das aresta do grafo G2 para negativo
for u, v, weight in G.edges(data="weight"):
    if weight is not None:
        G2[u][v]["weight"] = -weight

In [45]:
maior_caminho = nx.dijkstra_path(G2, S, L) # Usa Dijkstra com pesos invertidos
peso_maior_caminho = nx.dag_longest_path_length(G)  # devolve o maior acúmulo de pesos do grafo

In [46]:
# Imprime o caminho mínimo
vertices = {
    S:   "S",
    A:   "A",
    B:   "B",
    C:   "C",
    D:   "D",
    E:   "E",
    F:   "F",
    G_v: "G",
    H:   "H",
    I:   "I",
    J:   "J",
    K:   "K",
    L:   "L",
}
for v in maior_caminho:
    print(f"{vertices[v]} -> ", end="")      # O S e o FIM são parte da representação
print("FIM")
print(f"Peso do caminho: {peso_maior_caminho}")

S -> D -> G -> J -> K -> L -> FIM
Peso do caminho: 44


### O que sabemos até agora
Com o caminho de maior peso, também chamado de **caminho crítico**, encontrado acima, podemos definir que os vértices pertencentes a ele são essenciais para o menor tempo de execução, portanto, não podem se dispor a folgas. Assim temos:
* O tempo mínimo para realização da atividade é $44$
* As tarefas que não podem ter folgas são $D(4), G(7), J(10), K(11), L(12)$

Agora vamos encontrar as folgas para cada tarefa restante. Para encontrar essas folgas, calculamos o tempo de cada caminho indo em direção à última tarefa somando os tempos, depois o inverso, subtraindo do tempo crítico encontrado (44). Utilizaremos o algoritmo **BFS** para nos auxiliar. Vamos começar pelo **tempo de ida**, na direção de $S$ até $L$.

In [47]:
tempo_de_ida = [0 for i in range(13)]

print(" === Arestas Percorridas no BFS === ")
for edge in nx.edge_bfs(G, S):                                    # BFS da linha 5 + For da linha 6
    peso = G.get_edge_data(edge[0], edge[1])["weight"]
    soma = tempo_de_ida[edge[0]] + peso
    tempo_de_ida[edge[1]] = max(soma, tempo_de_ida[edge[1]])      # W = tempo_de_ida[U] + w(U, W)
    print(f"({vertices[edge[0]]} -> {vertices[edge[1]]}) = {peso}")

print("\n === Tempos de Ida === ")
for v in vertices:
    print(f"{vertices[v]} = {tempo_de_ida[v]}")

 === Arestas Percorridas no BFS === 
(S -> C) = 10
(S -> A) = 8
(S -> D) = 12
(S -> B) = 8
(C -> E) = 8
(C -> F) = 11
(A -> E) = 8
(A -> F) = 11
(A -> G) = 15
(D -> G) = 15
(B -> L) = 7
(E -> H) = 9
(F -> I) = 7
(G -> J) = 4
(H -> J) = 4
(I -> K) = 6
(J -> K) = 6
(K -> L) = 7

 === Tempos de Ida === 
S = 0
A = 8
B = 8
C = 10
D = 12
E = 18
F = 21
G = 27
H = 27
I = 28
J = 31
K = 37
L = 44


Agora veremos o **tempo de volta**! Vamos inverter as arestas do Dígrafo e realizar o processo de subtração para cada vértice, agora na direção $L$ até $S$.

In [48]:
G_revertido = G.reverse()                                   # Reverte as arestas Grafo

tempo_de_volta = [sys.maxsize for i in range(13)]           # Enche o array com "infinitos"
tempo_de_volta[L] = tempo_de_ida[L]                         # tempo_de_volta[L] = 44

for i in nx.edge_bfs(G_revertido, L):                       # BFS da linha 12 + For da linha 13
    peso = G_revertido.get_edge_data(i[0], i[1])["weight"]
    # Soma o próximo com o anterior
    soma = tempo_de_volta[i[0]] - peso                      # W = tempo_de_volta[U] - w(U, W)
    tempo_de_volta[i[1]] = min(soma, tempo_de_volta[i[1]]) 
    print(f"({vertices[i[0]]} -> {vertices[i[1]]}) = {peso}\tSoma = {soma}")

print("\n === Tempos de Volta === ")
for v in vertices:
    print(f"{vertices[v]} = {tempo_de_volta[v]}")

(L -> B) = 7	Soma = 37
(L -> K) = 7	Soma = 37
(B -> S) = 8	Soma = 29
(K -> I) = 6	Soma = 31
(K -> J) = 6	Soma = 31
(I -> F) = 7	Soma = 24
(J -> G) = 4	Soma = 27
(J -> H) = 4	Soma = 27
(F -> C) = 11	Soma = 13
(F -> A) = 11	Soma = 13
(G -> A) = 15	Soma = 12
(G -> D) = 15	Soma = 12
(H -> E) = 9	Soma = 18
(C -> S) = 10	Soma = 3
(A -> S) = 8	Soma = 4
(D -> S) = 12	Soma = 0
(E -> C) = 8	Soma = 10
(E -> A) = 8	Soma = 10

 === Tempos de Volta === 
S = 0
A = 10
B = 37
C = 10
D = 12
E = 18
F = 24
G = 27
H = 27
I = 31
J = 31
K = 37
L = 44


Agora, com esses dados, vamos calcular as respectivas folgas, dadas pela fórmula abaixo
$$folga = \textit{tempo\_de\_volta}[V] - \textit{tempo\_de\_ida}[V] (\forall V \in G)$$

In [49]:
folgas = {}
for v in vertices:
    folgas[vertices[v]] = tempo_de_volta[v] - tempo_de_ida[v]
    print(f"Folga de {vertices[v]} = {tempo_de_volta[v] - tempo_de_ida[v]} ")
print(folgas)

Folga de S = 0 
Folga de A = 2 
Folga de B = 29 
Folga de C = 0 
Folga de D = 0 
Folga de E = 0 
Folga de F = 3 
Folga de G = 0 
Folga de H = 0 
Folga de I = 3 
Folga de J = 0 
Folga de K = 0 
Folga de L = 0 
{'S': 0, 'A': 2, 'B': 29, 'C': 0, 'D': 0, 'E': 0, 'F': 3, 'G': 0, 'H': 0, 'I': 3, 'J': 0, 'K': 0, 'L': 0}


## Resultados
Com isso, encontramos todos os vértices e suas respectivas folgas, representadas pela tabela abaixo:


In [50]:
import pandas as pd
folgas.items()
pd.DataFrame(list(folgas.items()), columns=["Vértice", "Folga"])

,Vértice,Folga
0,S,0
1,A,2
2,B,29
3,C,0
4,D,0
5,E,0
6,F,3
7,G,0
8,H,0
9,I,3
